## Imports

### Stock Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
from time import sleep

In [3]:
os.getcwd()

'C:\\Users\\Diego\\Documents\\Git_code\\Brian\\RFMath\\Code'

In [4]:
import importlib

In [5]:
import numpy as np
import scipy as sp
from scipy.optimize import root
from scipy.interpolate import interp2d
import itertools
import time

In [6]:
import PIL

In [7]:
from scipy.ndimage import gaussian_filter
from scipy import interpolate

In [8]:
import bokeh
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
output_notebook()
from bokeh.palettes import Dark2
bokeh.io.curdoc().theme = 'dark_minimal'
palette = Dark2[8]*10

Loading BokehJS ...

In [9]:
palette = Dark2[8]*10
colors = itertools.cycle(palette)

In [10]:
from UtilityMath import plotComplexArray

Loading BokehJS ...

In [11]:
import skrf as rf

In [12]:
from scipy.optimize import minimize

### Custom Imports

In [13]:
from NetworkBuilding import (BuildMillerNetwork, BuildNewNetwork,
                             MillerMultLocsX, MillerCoupLocsX, NewMultLocs,
                             ConvertThetaPhiToTcX, 
                             Build3dBCoupler, Build5PortSplitter)

In [14]:
from ExpComponents import (Multiplier, MultiplierBank, Build3dBCouplerSim)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Loading BokehJS ...

dataSetRough.shape (35, 94, 94)


In [15]:
from Miller import (MillerBuilder)

In [16]:
from UtilityMath import (convertArrayToDict, MatrixError, MatrixSqError, makePolarPlot, addMatrixDiff, PolarPlot, ReIm)

In [17]:
from HardwareComms import (MultBankComm, SwitchComm, VNAComm, ExperimentalSetup)

COM3                
COM4                
COM5                


3 ports found


# New Architecture

This analysis has two steps: Tuning and Performance 

In the first step, we will tune the model of the devices to account for various idiosynchrasies of the physical network (device irregularities, cable lengths, etc).  

In order to do this tuning, we will build a simulation of the network.  The key element in the network is the Multiplier.  This element is represented by a Python Object that is based on a PCA analysis of physical measurements of large set of Multipliers.  Each Multiplier's representation has its own PCA weights that can be adjusted.

In order to tune these PCA weights, we will apply a series of test settings (PS value [0-1023] and VGA value [0-1023] to the Multipliers.  Upon performing a physical measurement, this will yield a series of $n \times n$ scattering matrices for the entire network.  Following that, we can use optimization to adjust the PCA weights of each Multiplier until the network simulations of the same test settings match the physical reasults.

Once the devices have been tuned, we can specifiy a desired target network response.  This network can be transformed into Multiplier complex transmission values, $T$.  The algorithm for this step can be quite complicated depending on the network topology (Miller vs New).  By using inverse functions on the PCA weights, we can find the required digital inputs (PS and VGA value) to each physical multiplier.

Finally, we apply these digital inputs both in simulation and experiment.  We take a physical measurement of the network and compare the target, simulation, and physical network responses.

## Definitions (Exp)

First we define the various devices.

In [35]:
inputSwitchComm = SwitchComm(comValue='COM4', portAliases={1:6, 2:5, 3:4, 4:3, 5:2, "test":1})
outputSwitchComm = SwitchComm(comValue='COM3', portAliases={1:3, 2:4, 3:5, 4:6, 5:7, "test":1})
vnaComm = VNAComm()
multBankComm = MultBankComm(comValue='COM5')

In [19]:
# inputSwitchComm.setSwitch("test", verbose=True)

In [20]:
# outputSwitchComm.setSwitch("test", verbose=True)

In [36]:
exp = ExperimentalSetup(inputSwitchComm, outputSwitchComm, multBankComm, vnaComm)

In [20]:
# switchCommIn = SwitchComm(comValue='COM1', {1:6, 2:5, 3:4, 4:3, 5:2})
# switchCommOut = SwitchComm(comValue='COM2', {1:2, 2:3, 3:4, 4:5, 5:6})
# vnaComm = VNAComm()
# multBankCom = MultBankComm(comValue='COM3')

For convenience, higher level scripts that require coordination between the various devices can be accessed using an `ExperimentalSetup`.

In [21]:
# exp = ExperimentalSetup(switchCommIn, switchCommOut, vnaComm, multBankCom)

## Definitions (Sim)

In [22]:
freq45 = rf.Frequency(start=45, stop=45, npoints=1, unit='mhz', sweep_type='lin')

First we need to generate labels for the Multipliers.  For the New architecture, this is a simple square grid.  The format is

`('M', 'N', inputLine, outputLine)` 

where `'M'` is for "Multiplier", `'N'` is for "New" and `inputLine` and `outputLine` are integers in the range [0,4].

In [23]:
allMultLocs = NewMultLocs(5,'N')
allMultLocs;

Every device has a "Physical Number" that is used for addressing to allow the computer to specify to which device a command is intended.  These are enumarated below.  Similar to SParams, the rows denote output lines while the columns denote input lines.

In [24]:
# Be careful here.  A horizontal row in the physical world represents a column in matrix multiplication
multPhysNumberBank = [[ 31, 32, 33, 34, 35],
                      [ 11, 12, 13, 14, 15],
                      [ 16, 17, 18, 19, 20],
                      [ 21, 22, 23, 24, 25],
                      [ 26, 27, 28, 29, 30]]
multPhysNumberBank = np.array(multPhysNumberBank).T
multPhysNumberBank

array([[31, 11, 16, 21, 26],
       [32, 12, 17, 22, 27],
       [33, 13, 18, 23, 28],
       [34, 14, 19, 24, 29],
       [35, 15, 20, 25, 30]])

And just a quick spot check to make sure we have accidently applied a transpose.

In [27]:
inputLine = 5
outputLine = 1
multPhysNumberBank[outputLine - 1, inputLine - 1]

26

Next we build a MultiplierBank.  This is a collection of Multipliers.  This allows a Multiplier to be retreived by either its `loc` or by its `physNumber`, allowing the MultiplierBank to function both to interact with the physical experiment or a network simulation.

In [26]:
multBank = MultiplierBank()
for loc in allMultLocs:
    (_, _, inputLine, outputLine) = loc
    physNumber = multPhysNumberBank[outputLine, inputLine]
    mult = Multiplier(physNumber=physNumber, loc=loc, freq=freq45)
    multBank.addMult(mult)

Note that passive devices such as 5:1 Splitters are not modeled to the same degree and do not require controlling.  Therefore, we will generate generic elements as we need them.

In [27]:
X0 = multBank.getPersonalityVectors()

## Tuning

### Debugging

In [28]:
for loc in allMultLocs:
    mult = multBank.getMultByLoc(loc)
    try: 
        multBankComm.blinkMult(mult.physNumber)
    except NameError:
        pass        
    sleep(0.2)

In [40]:
outIndex = 5
inIndex = 3
vga, ps = (1000, 100)
loc = ('M', 'N', inIndex-1, outIndex-1) # ('M', 'N', in, out) :(.
mult = multBank.getMultByLoc(loc)
physNum = mult.physNumber
print(physNum)
multBankComm.setMult(physNum, vga, ps)
inputSwitchComm.setSwitch(inIndex)
outputSwitchComm.setSwitch(outIndex)
sleep(2)
vnaComm.getS21AllAt45()

20


((-0.44238400668842964+1.3733359525934103j), 0.18244790504144695)

In [ ]:
multBankComm.setMult(28, 100, 200)

In [32]:
# inputSwitchComm.portAliases=None
# outputSwitchComm.portAliases=None

In [37]:
inputSwitchComm.setSwitch(1, verbose=True)
outputSwitchComm.setSwitch(1, verbose=True)
exp.vnaComm.getS21AllAt45()

MeshPort: 6
SwitchPort: 6
binary (CBA): 101


MeshPort: 3
SwitchPort: 3
binary (CBA): 010




((0.8337920047051187-0.7075025501147619j), 0.14538767911099154)

In [34]:
inputSwitchComm.close()
outputSwitchComm.close()

In [74]:
outputSwitchComm.setSwitch("test")

In [ ]:
exp.setMults(0, 100, multBank.getPhysNums())

In [75]:
exp.vnaComm.getS21AllAt45()

((1.0987181455856834e-05+0.00020863750964901913j), 0.0006027727154778587)

In [ ]:
SMat, STD = exp.measureSMatrix(delay=2)

In [ ]:
np.abs(SMat)

### Physical Measurement

Next we define a series of multiplier set points that we'll use to ascertain the multiplier's PCA weights.

In [ ]:
tuningPSVals = np.linspace(0, 1023, 10, dtype=np.int)
tuningVGAVals = np.linspace(0, 1023, 10, dtype=np.int)

In [ ]:
tuningVals = [(ps, vga) for vga in tuningVGAVals for ps in tuningPSVals]

For each PS, VGA pair, the multipliers are uniformly set and the scattering matrix of the network is measured.

In [ ]:
tuningMatricesM = []
for (psVal, vgaVal) in tuningVals:
    exp.setMults(int(psVal), int(vgaVal), multBank.getPhysNums())
    time.sleep(1)
    m, std = exp.measureSMatrix(delay=2)
    tuningMatricesM.append(m)
tuningMatricesM = np.array(tuningMatricesM)

In [ ]:
np.save("tuningVals10", tuningVals)
np.save("tuningMatricesM10", tuningMatricesM)

### Fake Measurements

In [ ]:
def MultBuilder(loc):
    return multBank.getRFNetwork(loc)

In [ ]:
def SplitterBuilder(loc):
    return Build5PortSplitter(freq45, loc=loc)

In [ ]:
X0 = multBank.getPersonalityVectors()

In [ ]:
XSet = X0*np.random.normal(1, 0.1, size=len(X0))

In [ ]:
multBank.setPersonalityVectors(XSet)

In [ ]:
tuningMatricesM = []
for (psVal, vgaVal) in tuningVals:
    multBank.setAllMults(psVal, vgaVal)
    newNet = BuildNewNetwork(SplitterBuilder, MultBuilder, loc="N", n=5)
    m = newNet.s[0, 5:, :5]
    tuningMatricesM.append(m)
tuningMatricesM = np.array(tuningMatricesM)

In [ ]:
multBank.setPersonalityVectors(X0)

In [ ]:
np.save("tuningVals", tuningVals)
np.save("tuningMatricesM", tuningMatricesM)

### Fitting

In [64]:
tuningVals = np.load("tuningVals10.npy")
tuningMatricesM = np.load("tuningMatricesM10.npy")

In [65]:
def PlotTuningMatrices(tuningMatrices, shape, maxRad):
    """
    tuningMatrices.shape => (N*M, n, n)
    shape = (N, M, n, n)
    """
    N, M, n, n = shape
    tuningMatricesNxN = tuningMatrices.reshape(shape)
    tuningMatricesNxN_List = [[tuningMatricesNxN[r,c] for c in range(M)] for r in range(N)]
    tuningMatrices2D = np.block(tuningMatricesNxN_List)
    plotComplexArray(tuningMatrices2D, maxRad=maxRad)

In [69]:
tuningVals[-1]

array([1023, 1023])

In [72]:
tuningMatricesM[-1]

array([[-0.505+1.392j, -0.553+1.478j, -0.812+1.434j, -0.526+1.267j, -0.287+1.395j],
       [-0.337+1.251j, -0.499+1.256j, -0.723+1.452j, -0.591+1.434j, -0.416+1.412j],
       [-0.174+1.548j, -0.215+1.514j, -0.382+1.374j, -0.266+1.535j, -0.271+1.431j],
       [-0.423+1.449j, -0.129+1.245j, -0.651+1.491j, -0.353+1.293j, -0.507+1.422j],
       [-0.389+1.322j,  0.041+1.224j, -0.639+1.45j , -0.358+1.405j, -0.504+1.394j]])

In [73]:
PlotTuningMatrices(tuningMatricesM, (10, 10, 5, 5), maxRad=1.5)

The simulation builder `BuildNewNetwork` requires that we supply it with two functions, one which creates an RF network object from of a 5-way splitter, and another which creates one of the Multiplier.  We will assume that the splitter is generic and employ a simple theoretical model for that which was imported from our `NetworkBuilding` theoretical simulation notebook.  However, for the Multiplier, we will use the `MultiplierBank` and the `loc` code to extract the model for a multiplier assigned to that specific location in the network. 

In [ ]:
def MultBuilder(loc):
    return multBank.getRFNetwork(loc)

In [ ]:
def SplitterBuilder(loc):
    return Build5PortSplitter(freq45, loc=loc)

As a quick example of a simulation, we set all the multipliers to the same setting, build a network, and examine the transmissive properties of it.

In [ ]:
multBank.setAllMults(psVal=512, vgaVal=512)

In [ ]:
newNet = BuildNewNetwork(SplitterBuilder, MultBuilder, loc="N", n=5)
T = newNet.s[0, 5:, :5]
T

Of course this step can be automated for all of the `(ps, vga)` pairs in the in `tuningVals` to yield `tuningMatricesS`.  

In [ ]:
tuningMatricesS = []
for (psVal, vgaVal) in tuningVals:
    multBank.setAllMults(psVal, vgaVal)
    newNet = BuildNewNetwork(SplitterBuilder, MultBuilder, loc="N", n=5)
    m = newNet.s[0, 5:, :5]
    tuningMatricesS.append(m)
tuningMatricesS = np.array(tuningMatricesS)

In [ ]:
PlotTuningMatrices(tuningMatricesS, (10, 10, 5, 5), maxRad=2.5)

Ideally, this would yield the exact same network scattering matrices as were measured and contained in `tuningMatricesM`.  Of course they won't because each physical device has its own personality and other factors such as varying cable lengths.  We will therefore optimize the PCA weights of each device in simulation in an attempt to create collection of devices which match the real behavior of the experimental devices.

In order to perform this optimization, we use SciPy's multivariate minimization function `minimize()`.  The format of this 
`scipy.optimize.minimize(fun, X0)` where `fun` is built such that `fun(X) -> error` where `X` and `X0` are 1D vectors of the real scalars to be optimized.  In order to make this easy, the MultiplierBank comes with two functions `setPersonalityVectors(X)` and `X0 = getPersonalityVectors()`, which grabs the complex PCA weights from all the multipliers as mashes them into a real 1D vector.  The two functions are designed to operate together so that the data

In [ ]:
X0 = multBank.getPersonalityVectors()

In [ ]:
def fun(X):
    multBank.setPersonalityVectors(X)
    tuningMatricesS = []
    for (psVal, vgaVal) in tuningVals:
        multBank.setAllMults(psVal, vgaVal)
        newNet = BuildNewNetwork(SplitterBuilder, MultBuilder, loc="N", n=5)
        m = newNet.s[0, 5:, :5]
        tuningMatricesS.append(m)
    tuningMatricesS = np.array(tuningMatricesS)
    error = np.sum(np.abs(tuningMatricesS - tuningMatricesM)**2)
    print(error)
    return error

In [ ]:
fit = sp.optimize.minimize(fun, X0, method='Powell', 
                           options={'disp':True, 'adaptive':True, 'fatol':0.01})

In [ ]:
XF = multBank.getPersonalityVectors()

In [ ]:
# XF = fit.x

Error when multipliers are the uniform average all devices measured in the PCA:

In [ ]:
fun(X0)

Error following fitting the PCA weights:

In [ ]:
fun(XF)

In [ ]:
multBank.setPersonalityVectors(XF)

In [ ]:
tuningMatricesS = []
for (psVal, vgaVal) in tuningVals:
    multBank.setAllMults(psVal, vgaVal)
    newNet = BuildNewNetwork(SplitterBuilder, MultBuilder, loc="N", n=5)
    m = newNet.s[0, 5:, :5]
    tuningMatricesS.append(m)
tuningMatricesS = np.array(tuningMatricesS)

In [ ]:
PlotTuningMatrices(tuningMatricesS, (10, 10, 5, 5), maxRad=2.5)

In [ ]:
np.save("personalityVector", XF)

# Set and Measure a Matrix

In [87]:
def calcNewMatrixSettings(K, multBank, n):
    expK = []
    for i_out in range(n):
        expRow = []
        for i_in in range(n):
            loc = ('M', 'N', i_in, i_out)
            mult = multBank.getMultByLoc(loc)
            T = 5*K[i_out, i_in]
            mult.setT(T)
            Texp = mult.TExpected
            expRow.append(Texp)
        expK.append(expRow)
    expK = np.array(expK)
    print(expK)

In [88]:
def setExpMultBank(exp, multBank):
    physNums = multBank.getPhysNums()
    psSettings = [multBank.getMultByPhysNum(physNum).psSetting for physNum in physNums]
    vgaSettings = [multBank.getMultByPhysNum(physNum).vgaSetting for physNum in physNums]
    exp.setMults(psSettings, vgaSettings, physNums)

In [89]:
XF = np.load("personalityVector.npy")

In [101]:
multBank.setPersonalityVectors(XF)

In [120]:
K = np.full((5,5), fill_value=(0.5+.5j))
K

array([[0.5+0.5j, 0.5+0.5j, 0.5+0.5j, 0.5+0.5j, 0.5+0.5j],
       [0.5+0.5j, 0.5+0.5j, 0.5+0.5j, 0.5+0.5j, 0.5+0.5j],
       [0.5+0.5j, 0.5+0.5j, 0.5+0.5j, 0.5+0.5j, 0.5+0.5j],
       [0.5+0.5j, 0.5+0.5j, 0.5+0.5j, 0.5+0.5j, 0.5+0.5j],
       [0.5+0.5j, 0.5+0.5j, 0.5+0.5j, 0.5+0.5j, 0.5+0.5j]])

In [121]:
calcNewMatrixSettings(K, multBank, 5)

[[2.498+2.498j 2.511+2.499j 2.497+2.502j 2.491+2.508j 2.497+2.5j  ]
 [2.489+2.506j 2.499+2.507j 2.491+2.499j 2.499+2.507j 2.507+2.485j]
 [2.489+2.507j 2.488+2.508j 2.501+2.505j 2.498+2.503j 2.51 +2.488j]
 [2.502+2.501j 2.489+2.506j 2.493+2.502j 2.495+2.513j 2.5  +2.507j]
 [2.5  +2.497j 2.497+2.508j 2.506+2.488j 2.51 +2.5j   2.51 +2.499j]]


In [122]:
multPhysNumberBank

array([[31, 11, 16, 21, 26],
       [32, 12, 17, 22, 27],
       [33, 13, 18, 23, 28],
       [34, 14, 19, 24, 29],
       [35, 15, 20, 25, 30]])

In [105]:
testMult = multBank.getMultByPhysNum(31)

In [106]:
(testMult.TExpected,
 testMult.vgaSetting,
 testMult.psSetting)

((-0.013212932769587259+2.4984706419659544j), 816, 369)

In [123]:
setExpMultBank(exp, multBank)

In [124]:
m, std = exp.measureSMatrix(delay=2)

In [125]:
mNew = m
mNew

array([[0.576+0.507j, 0.588+0.49j , 0.532+0.46j , 0.572+0.504j, 0.57 +0.511j],
       [0.582+0.5j  , 0.577+0.5j  , 0.528+0.452j, 0.582+0.498j, 0.578+0.503j],
       [0.578+0.501j, 0.583+0.498j, 0.533+0.457j, 0.574+0.502j, 0.569+0.506j],
       [0.584+0.497j, 0.494+0.6j  , 0.525+0.45j , 0.583+0.498j, 0.576+0.508j],
       [0.581+0.49j , 0.405+0.696j, 0.531+0.441j, 0.581+0.501j, 0.566+0.514j]])

In [126]:
np.abs(mNew - K)

array([[0.076, 0.089, 0.051, 0.072, 0.071],
       [0.082, 0.077, 0.056, 0.082, 0.078],
       [0.078, 0.083, 0.055, 0.074, 0.07 ],
       [0.084, 0.1  , 0.056, 0.083, 0.077],
       [0.082, 0.218, 0.066, 0.081, 0.068]])

In [128]:
plotData = np.hstack((K,mOld,mNew))

In [129]:
plotComplexArray(plotData, maxRad=1)

In [100]:
mOld = m
mOld

array([[0.391+0.32j , 0.437+0.328j, 0.338+0.404j, 0.348+0.31j , 0.408+0.266j],
       [0.395+0.207j, 0.333+0.32j , 0.359+0.379j, 0.412+0.338j, 0.413+0.287j],
       [0.481+0.24j , 0.47 +0.235j, 0.373+0.257j, 0.471+0.255j, 0.427+0.251j],
       [0.438+0.272j, 0.368+0.267j, 0.374+0.359j, 0.388+0.248j, 0.406+0.325j],
       [0.38 +0.274j, 0.348+0.269j, 0.375+0.339j, 0.425+0.26j , 0.375+0.341j]])

In [40]:
np.abs(K)

array([[0.361, 0.361, 0.361, 0.361, 0.361],
       [0.361, 0.361, 0.361, 0.361, 0.361],
       [0.361, 0.361, 0.361, 0.361, 0.361],
       [0.361, 0.361, 0.361, 0.361, 0.361],
       [0.361, 0.361, 0.361, 0.361, 0.361]])

# Scrap

In [ ]:
tuningMatricesS = []
for (psVal, vgaVal) in tuningVals:
    multBank.setAllMults(psVal, vgaVal)
    newNet = BuildNewNetwork(SplitterBuilder, MultBuilder, loc="N", n=5)
    m = newNet.s[0, 5:, :5]
    tuningMatricesS.append(m)
tuningMatricesS = np.array(tuningMatricesS)

tuningMatricesS - 

In [ ]:
physMatrices = []
for (psVal, vgaVal) in tuningVals:
    SetAllSimMults(psVal, vgaVal, multBank)
    time.sleep(1)
    m = MeasurePhysMatrix(5, inSwitchComm, outSwitchComm, vnaComm, delay=0)
    physMatrices.append(m)

In [ ]:
for loc in multBank.getLocs():
    mult = multBank.getMultByLoc(loc)
    mult.setSettings(psSetting, vgaSetting)

In [ ]:
mult = multBank.getMultByLoc(loc)
mult.setSettings(psSetting=0, vgaSetting=0)

In [ ]:
SplitterBuilder(("Sin", 0, 0))

In [ ]:
MultBuilder(("M", "X", 0, 0))

In [ ]:
np.allclose(T, Ks)

In [ ]:
freq = rf.Frequency(start=45, stop=45, npoints=1, unit='mhz', sweep_type='lin')

In [ ]:
def SplitterBuilder(loc):
    return Build5PortSplitter(freq, loc=loc)

In [ ]:
SplitterBuilder(("Sin", 0, 0))

In [ ]:
def MultBuilder(loc):
    (_, locParent, i_in, i_out) = loc
    Tc = Ks[i_out, i_in] * np.sqrt(5)**2
    return BuildMultiplier(Tc, freq, loc)

In [ ]:
MultBuilder(("M", "X", 0, 0))

In [ ]:
newNet = BuildNewNetwork(SplitterBuilder, MultBuilder, loc="X", n=5)
T = newNet.s[0, 5:, :5]
T

In [ ]:
np.allclose(T, Ks)

In [ ]:
Ks = np.array([[-0.05+0.06j, -0.  -0.13j, -0.07-0.15j,  0.11+0.28j, -0.05-0.18j],
               [-0.1 -0.19j, -0.3 -0.05j, -0.28+0.07j, -0.25+0.28j, -0.11-0.29j],
               [ 0.21-0.18j, -0.08-0.14j,  0.03+0.2j , -0.23+0.24j, -0.06+0.32j],
               [-0.29-0.31j,  0.12+0.09j,  0.08-0.02j,  0.31+0.12j, -0.22-0.18j],
               [-0.18-0.06j,  0.08-0.21j,  0.25-0.18j, -0.26-0.1j ,  0.13+0.1j ]])